In [1]:
import os
import random
import numpy as np
import pandas as pd
from scipy import stats, interpolate

In [7]:
def fix_distribution(x, y):
    indices=[0]
    for i in range(1,len(y)):
        if y[i] != y[i-1]:
            indices.append(i)
    x, y = x[indices], y[indices]
    return x, y


def get_inverse(args, query, n_bins=100, log=False):
    kernel = stats.gaussian_kde(query.period.values)
    lower, upper = min(query.period.values), max(query.period.values)
    if args.log:
        x = np.logspace(np.log10(lower),np.log10(upper),args.n_bins)
    else:
        x = np.linspace(lower,upper,args.n_bins)
    y = np.cumsum(kernel(x))/np.sum(kernel(x))
    x, y = fix_distribution(x, y)
    try:
        spline = interpolate.CubicSpline(y, x)
    except ValueError:
        return None, None
    else:
        xnew = np.linspace(min(y),max(y),args.n_bins)
        return xnew, spline
    
# Main function to import when not using CLI
def get_period(teff, logg, path='rotation.csv', min_sample=20, res_teff=100., res_logg=0.1, log=False, n_bins=100, verbose=True):
    period=[]
    # read in known rotation periods and get limits
    df = pd.read_csv(path)
    for tt, ll in zip(teff, logg):
        per = np.nan
        # select stars near target in HR diagram
        query = df.query("teff >= %f and teff < %f and logg >= %f and logg < %f"%(tt-(res_teff/2.), tt+(res_teff/2.), ll-(res_logg/2.), ll+(res_logg/2.)))
        if len(query) < min_sample:
            if verbose:
                print('WARNING: not enough in the sample to create an accurate distribution.\nTry changing the resolution of the grid to include more stars!')
                print('Currently using teff +/- %.1f K and logg +/- %.2f dex'%(res_teff/2.,res_logg/2.))
        else:
            kernel = stats.gaussian_kde(query.period.values)
            lower, upper = min(query.period.values), max(query.period.values)
            if log:
                x = np.logspace(np.log10(lower),np.log10(upper),n_bins)
            else:
                x = np.linspace(lower,upper,n_bins)
            y = np.cumsum(kernel(x))/np.sum(kernel(x))
            x, y = fix_distribution(x, y)
            try:
                spline = interpolate.CubicSpline(y, x)
            except ValueError:
                continue
            else:
                # draw random number to map back to period distribution
                per = spline(random.random())+0.
        period.append(per)
    return np.array(period)

### Example for a single star
if it's for one, just make them list-like
(because this is optimized for command line - which will append values to a list)

In [11]:
prot = get_period([5777.], [4.4])
print(prot)

[19.78076579]


### Running a handful of stars
**note:** make sure to reset the "period"

In [13]:
teff = [5250., 5777., 5250., 5858., 6000.]
logg = [4.6, 4.4, 4.5, 4.3, 4.2]

prot = get_period(teff, logg)
print(prot)

[34.4363753  26.43575009 11.1858871  10.97166953  0.50360504]
